In [1]:
# 实验使用不同的RNN结构，实现一个人脸图像分类器。
# 至少对比2种以上结构训练损失和准确率差异，如：LSTM、GRU、RNN、BiRNN等。要求使用tensorboard，
# 提交代码及run目录和可视化截图。
# https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_olivetti_faces.html#

In [2]:
# 导包
from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader,TensorDataset
from model.rnn_model import RNN
from model.lstm_model import LSTM
from model.gru_model import GRU
from torch.utils.tensorboard import SummaryWriter

In [3]:
# 超参数定义
BATCH_SIZE = 32
epochs = 30

In [4]:
# 1、数据集加载
images, label = fetch_olivetti_faces(data_home='./face_data', shuffle=True, return_X_y=True)

X = images.reshape(-1, 64, 64)  # (400, 64, 64)
y = label #(400,)
# 2、拆分测试集 (每个批次中的样本数量, 每个样本中时间步的数量, 每个时间步的特征数量)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

# 3、数据集转tensor (unsqueeze在1维添加一个维度) 
# X（320,4096）->(320,1,4096) Y（320,）->(320)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 4、捆绑输入数据和对应的标签
train_datasets = TensorDataset(X_train_tensor, y_train_tensor)
test_datasets = TensorDataset(X_test_tensor, y_test_tensor)

# 5、分批次
train_loader = DataLoader(train_datasets, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_datasets, batch_size=BATCH_SIZE, shuffle=True)

In [5]:
# 训练模型
def train_model(model, writer, model_name):
    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    for epoch in range(epochs):
        train_loss = 0 # 总损失值
        for X,y in train_loader:
            y_hat = model(X)
            loss = loss_fn(y_hat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        avg_loss = train_loss / len(train_loader) # 当前批次平均损失
        writer.add_scalar(f'{model_name}/Train Loss', avg_loss, epoch)
        print(f"epoch: {epoch}, loss: {avg_loss}")
    return avg_loss

In [6]:
# 定义测试验证函数
def test_train(model):
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for images, label in test_loader:
            y_hat = model(images)
            _, max_idx = torch.max(y_hat, dim=1)
            total += label.size(0)
            correct += (max_idx == label).sum().item()
        print(f"total: {total}, correct: {correct}")
    return correct/total * 100

In [7]:
# 训练
models = {
    "RNN": RNN(),
    'LSTM': LSTM(),
    'GRU': GRU()
}

writer = SummaryWriter('runs/face_classfier')

for model_name, model in models.items():
    print(f"Training {model_name}...")
    train_loss = train_model(model, writer, model_name)
    accuracy = test_train(model)
    writer.add_scalar(f'{model_name}/Test Accuracy', accuracy, 0)
    print(f"{model_name} Test Accuracy: {accuracy}%")
    
writer.close()

Training RNN...
epoch: 0, loss: 3.708177328109741
epoch: 1, loss: 3.5733075857162477
epoch: 2, loss: 3.3024811029434202
epoch: 3, loss: 3.0858227968215943
epoch: 4, loss: 2.9007876396179197
epoch: 5, loss: 2.678826355934143
epoch: 6, loss: 2.5816395759582518
epoch: 7, loss: 2.4255423545837402
epoch: 8, loss: 2.343341255187988
epoch: 9, loss: 2.209446680545807
epoch: 10, loss: 2.12331737279892
epoch: 11, loss: 2.0209432244300842
epoch: 12, loss: 1.9134453296661378
epoch: 13, loss: 1.8719324707984923
epoch: 14, loss: 1.8357917547225953
epoch: 15, loss: 1.8679407000541688
epoch: 16, loss: 1.8161273837089538
epoch: 17, loss: 1.7131071090698242
epoch: 18, loss: 1.6755007028579711
epoch: 19, loss: 1.7315719962120055
epoch: 20, loss: 1.552405846118927
epoch: 21, loss: 1.4836797952651977
epoch: 22, loss: 1.4075274348258973
epoch: 23, loss: 1.452639353275299
epoch: 24, loss: 1.329300320148468
epoch: 25, loss: 1.3174033164978027
epoch: 26, loss: 1.2849681615829467
epoch: 27, loss: 1.234694635868